In [1]:
import pandas as pd
import numpy as np
import gensim
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
df = pd.read_csv("yuz_veri_kumesi.csv", sep=";")
df.head()

,Cumle,Sinif
0,Onu bu yüzden suçlamıyordum.,6
1,Ciddi ifadeler taşıyan yüzlerine baktım.,2
2,Simmon’un yüz ifadesi dehşet içindeydi.,2
3,O yüzden aklımda kalmış olacak!,6
4,Sleat yüzünü ovuşturdu.,2


In [3]:
import re
cumle = []
for i in df['Cumle']:
    kelime = str(i)
    kelime = re.sub(r"\d+", " ", kelime)               # tum sayilari cikar
    kelime = re.sub(r"\b[a-zA-Z]\b", "", kelime)       # tüm yalniz karakterleri cikar
    kelime = re.sub(r"\b..\b", " ", kelime)            # iki karakterli kelimeleri cikar
    kelime = re.sub(r"[^\w\s]", " ", kelime)           # tum noktalama isaretlerini cikar
    kelime = re.sub(r"\s+", " ", kelime)               # cift boslukları tek boslukla degistirir
    cumle.append(kelime)

In [4]:
df['Cumle'] = cumle

In [5]:
tokens = df['Cumle'].apply(lambda x: gensim.utils.simple_preprocess(x))

In [6]:
X_train, X_test, y_train, y_test = train_test_split (tokens, df['Sinif'] , test_size=0.7)

In [7]:
w2v_model = gensim.models.Word2Vec(X_train)
kelimeler = set(w2v_model.wv.index_to_key )

In [8]:
X_train = np.array([np.array([w2v_model.wv[i] for i in cumle if i in kelimeler]) for cumle in X_train])
X_test = np.array([np.array([w2v_model.wv[i] for i in cumle if i in kelimeler]) for cumle in X_test])

C:\Users\seyma\AppData\Local\Temp\ipykernel_12100\963835519.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train = np.array([np.array([w2v_model.wv[i] for i in cumle if i in kelimeler]) for cumle in X_train])
C:\Users\seyma\AppData\Local\Temp\ipykernel_12100\963835519.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test = np.array([np.array([w2v_model.wv[i] for i in cumle if i in kelimeler]) for cumle in X_test])


In [9]:
X_train_avg = []
X_test_avg = []
for v in X_train:
    if v.size:
        X_train_avg.append(v.mean(axis=0))
    else:
        X_train_avg.append(np.zeros(100, dtype=float))

for v in X_test:
    if v.size:
        X_test_avg.append(v.mean(axis=0))
    else:
        X_test_avg.append(np.zeros(100, dtype=float))

In [10]:
nb = RandomForestClassifier(max_depth=20, random_state=0)
model = nb.fit(X_train_avg, y_train.values.ravel())

In [11]:
y_pred = model.predict(X_test_avg)

In [12]:
print("accuracy score:",accuracy_score(y_test, y_pred))
print("f1_score:",f1_score(y_test, y_pred, average='weighted'))

accuracy score: 0.713116670563479
f1_score: 0.6831691147300603
